In [1]:
from multiprocessing import Pool

In [2]:
#### add info for ASm6A
# asm6a_dir = "/Charles/project/ASm6A/ASm6A/"
asm6a_dir = "/Charles/project/ASm6A/Hypothesis_test/ASm6A/rename/reformat/"
sample_info = "/Charles/project/ASm6A/data/sample_info.txt"
result_dir = "/Charles/project/ASm6A/ASm6A/cal_variance/ASm6A/"
os.system("mkdir -p %s" % result_dir)

df = pd.read_table(sample_info, index_col=0)
sample_dict = {}
for sample, values in df.iterrows():
    sample_dict[sample] = ",".join(values.tolist())

for sample, info in sample_dict.items():
    asm6a = os.path.join(asm6a_dir, sample)
    res = os.path.join(result_dir, sample)
    df = pd.read_table(asm6a, header=None)
    df['info'] = info
    df.to_csv(res, sep="\t", header=False, index=False)

In [1]:
# #### split bed into bins (each bin contains one peak)
# os.chdir("/Charles/project/ASm6A/ASm6A/cumulative_peak/fetal/")
# result_dir = "bin_peaks/"
# os.system("mkdir -p %s" % result_dir)
# cumulative_peak = "fetal.bed"
# with open(cumulative_peak, 'r') as f:
#     for line in f.readlines():
#         res = os.path.join(result_dir, "_".join(line.strip().split("\t")) + ".bed")
#         with open(res, 'w') as fw:
#             fw.write(line)

In [3]:
#### select all ASm6As (across all individuals and tissues) for each m6A peak
#### 
# overlap with peak
# ##############
asm6a_dir = "/Charles/project/ASm6A/ASm6A/cal_variance/ASm6A/"
bed_list = glob.glob("%s/*.bed"%asm6a_dir)
cumulative_peak = "/Charles/project/ASm6A/ASm6A/cumulative_peak/fetal/fetal.bed"
##############
result_dir = "/Charles/project/ASm6A/ASm6A/cal_variance/overlap_peak/"
os.system("mkdir -p %s" % result_dir)
##
for bed in bed_list:
    res = os.path.join(result_dir, os.path.basename(bed))
    os.system("bedtools intersect -a %s -b %s -wa -wb > %s" % (bed, cumulative_peak, res))
##
os.chdir(result_dir)
total_res = "total_overlap.txt"
os.system("cat *.bed > %s" % total_res)


###########
# reformat 
##########
def write2file(df):
#     print(df.iloc[0,:])
    res = os.path.join(result_dir, "%s.txt" % df.iloc[0,-4])
    df['all_info'].to_csv(res, header=False, index=False)

os.chdir("/Charles/project/ASm6A/ASm6A/cal_variance/overlap_peak/")
result_dir = "reformat/"
os.system("mkdir -p %s" % result_dir)
df = pd.read_table("total_overlap.txt", header=None)
df['peak'] = df.iloc[:,-3] + "_" + df.iloc[:,-2].astype(str) + "_" + df.iloc[:,-1].astype(str)
df['info'] = df.iloc[:,6] ##### attention!
df['imbalance'] = np.abs(df.iloc[:,3].str.split(";").str[1].astype(float) - 0.5)
df['all_info'] = df['imbalance'].astype(str) + "," + df['info']
# print(df.head())
# print(df.iloc[0,:])
    
df.groupby(['peak']).apply(write2file)

""


In [4]:
### future filter
os.chdir("/Charles/project/ASm6A/ASm6A/cal_variance/overlap_peak/reformat/")
result_dir = "final/"
os.system("mkdir -p %s" % result_dir)
#
txt_list = glob.glob('*.txt')
for txt in txt_list:
    res = os.path.join(result_dir, txt)
    line_num = int(os.popen("wc -l %s" % txt).read().split()[0])
    if line_num >= 3:
        t = open(txt, 'r').read()
        # new_t = t.replace(' Weeks,Female"','').replace(' Weeks,Male"','').replace('"','')
        new_t = t.replace(' Weeks','').replace(' Weeks','').replace('"','')
        with open(res, 'w') as fw:
            fw.write(",".join(["allelic","individual","tissue","age","sex\n"]))
            fw.write(new_t)
        df = pd.read_csv(res)
        if len(set(df["allelic"].tolist())) < 2:
            os.system("rm %s" % res)
        for col in ["individual","tissue","age","sex"]:
            if (len(set(df[col].tolist())) < 2) | (len(set(df[col].tolist())) >= len(df)):
                os.system("rm %s" % res)

In [5]:
#### parse lmer4 results
os.chdir("/Charles/project/ASm6A/ASm6A/cal_variance/overlap_peak/reformat/final/results/")
result_dir = "variance_from_tissueORind/"
os.system("mkdir -p %s" % result_dir)
#
txt_list = glob.glob("*.txt")
result_list = []
for txt in txt_list:
    prefix = txt.split(".txt")[0]
    with open(txt, 'r') as f:
        contents = f.readlines()
        for i in range(len(contents)):
            if "Random effects:" in contents[i]:
                var_dict = {}
                for line in [contents[i+2], contents[i+3]]:
                    if "individual" in line:
                        var_dict["ind"] = line.split("individual (Intercept) ")[1].split(" ")[0]
                    elif "tissue" in line:
                        var_dict["tissue"] = line.split("tissue     (Intercept) ")[1].split(" ")[0]
                result_list.append([prefix, var_dict['ind'], var_dict['tissue']])
df = pd.DataFrame(result_list)
df.columns = ['m6A', 'Ind_var', 'Tissue_var']
print(df.head())
result_file = os.path.join(result_dir, "tissue_ind_variance.txt")
df.to_csv(result_file, sep="\t", index=False)

                        m6A    Ind_var Tissue_var
0   chr15_56958918_56959217   0.000000   0.007017
1   chr19_53959681_53960997   0.000000   0.000000
2  chr4_187524154_187525128  1.932e-05  2.420e-05
3    chr5_10760944_10761417   0.000000   0.000000
4  chr6_146125519_146127672   0.006309   0.000000
